In [1]:
import re
import operator
import networkx as nx

In [2]:
with open('../data/2024/day24.txt') as f:
    data = f.read()

In [3]:
wires, gates = data.split("\n\n")
wires = [tuple(w.split(': ')) for w in wires.splitlines()]
gates = [re.findall(r'(.*) (XOR|OR|AND) (.*) -> (.*)', g)[0] for g in gates.splitlines()]

In [4]:
G = nx.DiGraph()

# Initial wires
for w, v in wires:
    G.add_node(w)

# Add a pair of directed input edges to gates with one output
for i, (u, op, v, out) in enumerate(gates):
    gate_name = f'{op}{i}'
    G.add_node(gate_name, op=op)
    G.add_edge(u, gate_name)
    G.add_edge(v, gate_name)
    G.add_edge(gate_name, out)

# Outputs sorted in reverse order for bits
outputs = sorted(filter(lambda node: str(node).startswith('z'), G.nodes), reverse=True)

In [5]:
# Each gate has two inputs, though either can also be a gate
def get_gate_value(gate):
    inputs = G.predecessors(gate)
    a = get_node_value(next(inputs))
    b = get_node_value(next(inputs))

    if gate.startswith('XOR'):
        return operator.xor(a, b)
    elif gate.startswith('OR'):
        return operator.or_(a, b)
    elif gate.startswith('AND'):
        return operator.and_(a, b)
    else:
        raise ValueError('Unknown gate')

# Recursively determine the voltage of a node by looking at its inputs
def get_node_value(node):
    # This is a gate
    if G.nodes[node].get('op') in ['OR', 'AND', 'XOR']:
        return get_gate_value(node)
    else:
        # This isn't an initial input
        if list(G.predecessors(node)):
            return get_node_value(next(G.predecessors(node)))
        else: # This is an initial input
            return G.nodes[node].get('voltage', False)

In [6]:
# Part 1
# Add voltages to the initial wires as bools
for w, v in wires: G.nodes[w]['voltage'] = True if v == '1' else False

# Start at the outputs and recurse backwards to determine voltages
bits = ''.join(map(str,[1 if bit else 0 for bit in [get_node_value(output) for output in outputs]]))
print(f"Part 1: {int(bits, 2)}")

Part 1: 52038112429798
